In [46]:
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Lambda, Dense, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt

In [60]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)), 
    # (2,2) is the default pool size so we could have just used MaxPooling2D() with no explicit arguments
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)), 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units = 128, activation = 'relu'), # You can apply softmax activation here, see below for comentary
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [61]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [58]:
# Create a basic model instance
#model = create_model()

In [62]:
# Display the model's architecture
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 28800)             0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)               3686528   
_________________________________________________________________
dense_25 (Dense)             (None, 1)               

In [63]:
# re-size all the images to this
IMAGE_SIZE = [128, 128]

train_path = r'Covid/Train'
valid_path = r'Covid/Test'

In [64]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#In test data we don't do data augumentation

test_datagen = ImageDataGenerator(rescale = 1./255)

In [65]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(r'Covid/Train',
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 1877 images belonging to 2 classes.


In [66]:
test_set = test_datagen.flow_from_directory(r'Covid/Test',
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 735 images belonging to 2 classes.


In [67]:
model.fit(
  training_set,
  validation_data=test_set,
  epochs=3,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/3
59/59 [==============================] - 47s 781ms/step - loss: 0.1456 - accuracy: 0.9361 - val_loss: 0.0220 - val_accuracy: 0.9905
Epoch 2/3
59/59 [==============================] - 45s 756ms/step - loss: 0.0193 - accuracy: 0.9925 - val_loss: 0.0160 - val_accuracy: 0.9946
Epoch 3/3
59/59 [==============================] - 45s 759ms/step - loss: 0.0171 - accuracy: 0.9941 - val_loss: 0.0372 - val_accuracy: 0.9864


In [68]:
!mkdir -p saved_model

In [71]:
model.save('my_model.h5')

In [72]:
# Testing our model
test_loss, test_accuracy = model.evaluate(test_set)

23/23 [==============================] - 5s 223ms/step - loss: 0.0372 - accuracy: 0.9864


In [73]:
# Printing the test results
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.0372. Test accuracy: 98.64%


In [ ]:
#y_pred = model.predict(test_set)

In [ ]:
#import numpy as np
#y_pred = np.argmax(y_pred1, axis=1)

In [1]:
from keras.models import load_model
import numpy as np
from keras.preprocessing import image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Lambda, Dense, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt

In [2]:
model = load_model('my_model.h5')


In [20]:
#img_path= r'D:\Albot\Medical Imaging\Code\Covid\Test\Others\apple.jpg'
img_path= r'D:\Albot\Medical Imaging\Code\Covid\Test\Others\cat.840.jpg'
#img_path= r'D:\Albot\Medical Imaging\Code\Covid\Test\Others\dog.2487.jpg'
#img_path= r'D:\Albot\Medical Imaging\Code\Covid\Test\Lungs\COVID-111.png'

In [21]:
img= image.load_img(img_path, target_size=(128,128))
x=image.img_to_array(img)
x=x/255
x1=np.expand_dims(x,axis=0)


In [22]:
#from keras.applications.preprocess_input import preprocess_input
#img_data=preprocess_input(x1)
#img_data.shape

In [23]:
result = model.predict(x1)

In [24]:
result

array([[1.]], dtype=float32)

In [25]:
if result[0][0]>0.5:
    print('Others')
else:
    print('Covid')

Others
